<a href="https://colab.research.google.com/github/Slebbon/TextGeneration_Projet_PSL_EnC/blob/main/Bert_classifier_Shrump.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets
!pip install -q evaluate

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 65.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-uncased",fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#setup parameters regarding GPU availibility on the machine and recycle used memory
import torch;
import gc;

is_gpu_available = torch.cuda.is_available()
device = 'cuda' if is_gpu_available else 'cpu'
if is_gpu_available:
    print("GPU available for notebook")
    torch.cuda.empty_cache()
    print("GPU Memory cleaned")
else:
    print("No GPU available for notebook")

gc.collect()

GPU available for notebook
GPU Memory cleaned


135

In [ ]:
from datasets import ClassLabel,DatasetDict, load_dataset, Dataset

In [ ]:
pd.read_csv('/content/I_train.csv').dropna(subset='Text').rename(columns={'Author':'label'}).to_csv('/content/I_train.csv',index=False)
pd.read_csv('/content/I_validation.csv').dropna(subset='Text').rename(columns={'Author':'label'}).to_csv('/content/I_validation.csv',index=False)
pd.read_csv('/content/I_test.csv').dropna(subset='Text').rename(columns={'Author':'label'}).to_csv('/content/I_test.csv',index=False)

In [ ]:
dataset = load_dataset("/content/", data_files={
    'train': f'I_train.csv',
    'validation': f'I_validation.csv',
    'test': f'I_test.csv'
})

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset = dataset.cast_column("label", ClassLabel(num_classes=3, names=['Shakespeare','Trump','Other'])).shuffle(seed=10)

Casting the dataset:   0%|          | 0/32834 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7042 [00:00<?, ? examples/s]

Casting the dataset:   0%|          | 0/7044 [00:00<?, ? examples/s]

In [ ]:
id2label = {0:'Shakespeare',
            1: 'Trump',
            2:'Other'}

In [ ]:
label2id = {value:key for key,value in id2label.items()}

In [ ]:
def preprocess(exemple):
  return tokenizer(exemple['Text'],truncation=True, padding=False)

In [ ]:
ddic = dataset.map(preprocess,batched=True)

Map:   0%|          | 0/32834 [00:00<?, ? examples/s]

Map:   0%|          | 0/7042 [00:00<?, ? examples/s]

Map:   0%|          | 0/7044 [00:00<?, ? examples/s]

In [ ]:
from transformers import DataCollatorWithPadding

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)


In [ ]:
from transformers import TrainingArguments

In [ ]:
from peft import LoraConfig, TaskType

In [ ]:
lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS, r=16, lora_alpha=16, lora_dropout=0.1, bias='all',
)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased",
                                                           device_map=0,
                                                           id2label=id2label, label2id=label2id)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [ ]:
import evaluate
import numpy as np

def compute_BERT_classifier(eval_preds):

    # metrics for the classifier
    metric1 = evaluate.load("f1")
    metric2 = evaluate.load('accuracy')
    metric3 = evaluate.load('precision')
    metric4 = evaluate.load('recall')
    logits = eval_preds.predictions
    labels = eval_preds.label_ids

    # choose prediction with maximum probability from logits via argmax method
    # axis = -1, last dimension
    predictions = np.argmax(logits, axis=-1)
    f1 = metric1.compute(predictions=predictions, references=labels, average='micro')
    accuracy = metric2.compute(predictions=predictions, references=labels)
    precision = metric3.compute(predictions=predictions, references=labels, average='micro')
    recall = metric4.compute(predictions=predictions, references=labels, average='micro')

    return {"f1":f1['f1'], "accuracy":accuracy['accuracy'], "precision":precision['precision'], 'recall':recall['recall']}

In [ ]:
training_args = TrainingArguments(
    output_dir = "content/model/trainings/",
    logging_dir = "content/model/logs/",
    evaluation_strategy='steps',
    label_names = None,
    per_device_train_batch_size=8,
    num_train_epochs=3,
    load_best_model_at_end=True,
    save_steps=0.1,
    eval_steps=0.1,
    logging_steps=0.1,
    metric_for_best_model="eval_f1",
    learning_rate = 5e-04
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer

In [ ]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=ddic["train"],
    eval_dataset=ddic['validation'],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_BERT_classifier
)

In [ ]:
trainer.evaluate()

{'eval_loss': 1.071146011352539,
 'eval_f1': 0.35870491337688154,
 'eval_accuracy': 0.35870491337688154,
 'eval_precision': 0.35870491337688154,
 'eval_recall': 0.35870491337688154,
 'eval_runtime': 21.2058,
 'eval_samples_per_second': 332.079,
 'eval_steps_per_second': 41.545}

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall
1232,0.096100,0.056808,0.985373,0.985373,0.985373,0.985373
2464,0.064300,0.035140,0.991906,0.991906,0.991906,0.991906
3696,0.052800,0.027957,0.992474,0.992474,0.992474,0.992474
4928,0.039200,0.022432,0.994604,0.994604,0.994604,0.994604
6160,0.031500,0.053137,0.989776,0.989776,0.989776,0.989776
7392,0.026300,0.022574,0.994604,0.994604,0.994604,0.994604
8624,0.019800,0.026437,0.993468,0.993468,0.993468,0.993468
9856,0.011300,0.025501,0.994888,0.994888,0.994888,0.994888
11088,0.012200,0.029459,0.994888,0.994888,0.994888,0.994888


TrainOutput(global_step=12315, training_loss=0.03639785450646125, metrics={'train_runtime': 848.1384, 'train_samples_per_second': 116.139, 'train_steps_per_second': 14.52, 'total_flos': 2453276502706320.0, 'train_loss': 0.03639785450646125, 'epoch': 3.0})

In [ ]:
test_predictions = trainer.predict(ddic['test'])

In [ ]:
getattr(test_predictions, 'metrics')

{'test_loss': 0.03073740005493164,
 'test_f1': 0.9951731970471324,
 'test_accuracy': 0.9951731970471324,
 'test_precision': 0.9951731970471324,
 'test_recall': 0.9951731970471324,
 'test_runtime': 19.2464,
 'test_samples_per_second': 365.991,
 'test_steps_per_second': 45.775}

In [ ]:
eval_predictions = trainer.predict(ddic['validation'])

In [ ]:
getattr(eval_predictions, 'metrics')

{'test_loss': 0.020814990624785423,
 'test_f1': 0.9960238568588469,
 'test_accuracy': 0.9960238568588469,
 'test_precision': 0.9960238568588469,
 'test_recall': 0.9960238568588469,
 'test_runtime': 19.4539,
 'test_samples_per_second': 361.984,
 'test_steps_per_second': 45.287}

In [ ]:
trainer.save_model('classifier_shrump')

# Check trainer metrics for each class

In [ ]:
compute_BERT_classifier(test_predictions)

{'f1': 0.9947473026689381,
 'accuracy': 0.9947473026689381,
 'precision': 0.9947473026689381,
 'recall': 0.9947473026689381}

In [ ]:
def compute_metrics(eval_preds):

    # metrics for the classifier
    metric1 = evaluate.load('f1')
    metric2 = evaluate.load('precision')
    metric3 = evaluate.load('recall')
    logits = eval_preds.predictions
    labels = eval_preds.label_ids

    # choose prediction with maximum probability from logits via argmax method
    # axis = -1, last dimension
    predictions = np.argmax(logits, axis=-1)
    f1 = metric1.compute(predictions=predictions, references=labels, average=None)
    prec = metric2.compute(predictions=predictions, references=labels, average=None)
    rec = metric3.compute(predictions=predictions, references=labels, average=None)
    return {"f1":f1['f1'], 'precision': prec['precision'], 'recall':rec['recall']}

In [ ]:
compute_metrics(test_predictions)

{'f1': array([0.99554367, 0.99782892, 0.99346923]),
 'precision': array([0.99495249, 0.99739583, 0.99445325]),
 'recall': array([0.99613555, 0.99826238, 0.99248715])}

# Save model to drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp -r /content/classifier_shrump /content/drive/MyDrive

# Loading final model

In [ ]:
from transformers import pipeline

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-uncased",device_map=0,id2label=id2label, label2id=label2id)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from peft import PeftModel, PeftConfig

In [ ]:
model = PeftModel.from_pretrained(model, '/content/classifier_shrump')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("/content/classifier_shrump",fast=True, device_map=0, truncation=True)

In [ ]:
pipe = pipeline(model=model,tokenizer=tokenizer,task='text-classification',device_map=0)

The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassification', 'ElectraForSequenceClassification', 'ErnieForSequenceClassification', 'ErnieMForSequenceClassification', 'EsmForSequenceClassification', 'FalconForSequenceClassification', 'FlaubertForSequenceClassification', 'FNetForSequenceClassification', 'FunnelForSequenceClassification', 'GemmaForSequenceClassification'